# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [6]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2020-05-07	TOI-1338: TESS' First Transiting Circumbinary Planet	The Astronomical Journal	This paper is about the number 1. The number 2 is left for future work.	Veselin B. Kostov *et al* 2020 *AJ* **159** 253	TOI-1338	http://academicpages.github.io/files/paper1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [7]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [8]:
publications = pd.read_json("pubs.json").T

publications


,pub_date,title,body,venue,excerpt,citation,url_slug,paper_url,published
6,2023-06-15,"Formation, Structure, and Detectability of the...",I worked with Dr. Jamey Szalay to model the fo...,Planetary Science Journal,We present three posible formation models for ...,W. Z. Cukier and J. R. Szalay 2023 *Planet. Sc...,Geminids,https://doi.org/10.3847/PSJ/acd538,True
5,2022-05-01,Natural Transport of Microbes from Early Venus...,"I worked with Dr. Chris Chyba to estimate, giv...",Junior Paper,My Spring JP,,Venus-JP,,False
4,2023-01-01,Polarized Raytracing of Black Hole Magnetospheres,I worked with Dr. Benjamin Crinquand and Dr. A...,Junior Paper,My Fall JP,,Black-Hole-JP,,False
3,2023-05-12,ABM-2: An improved Absolute Beam Monitor,An absolute beam monitor (ABM) is a device use...,Class Presentation,This project was completed in the year long la...,,ABM,,False
2,2019-11-23,Habitable Zone Boundaries for Circumbinary Pla...,This was the research that I did the summer be...,Publications of the Astronomical Society of th...,We used a climate model to understand the habi...,Wolf Cukier *et al* 2019 *PASP* **131** 124402,CBP-HZ,https://arxiv.org/pdf/1911.02983.pdf,True
1,2020-05-07,TOI-1338: TESS' First Transiting Circumbinary ...,TOI-1338 b is the planet that I helped to disc...,The Astronomical Journal,This paper describes the discovery of the circ...,Veselin B. Kostov *et al* 2020 *AJ* **159** 253,TOI-1338,https://iopscience.iop.org/article/10.3847/153...,True


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
import os
dir = "../_research/"
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))
    
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /research/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    if item.published: md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.body)) > 5:
        md += "\n" + html_escape(item.body) + "\n"
    
    if item.published:
        if len(str(item.paper_url)) > 5:
            md += "\n[Download paper here](" + item.paper_url + ")\n" 
            
        md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_research/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [11]:
!ls ../_research/

2019-11-23-CBP-HZ.md        2023-01-01-Black-Hole-JP.md
2020-05-07-TOI-1338.md      2023-05-12-ABM.md
2022-05-01-Venus-JP.md      2023-06-15-Geminids.md


In [12]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
